# Deep Agents(LongChain-ai)
> https://github.com/langchain-ai/deepagents

## 测试一
- 学习使用 LLM 调用不同代理实现更通用的任务实现

    - 规划工具

    - 子代理

    - 文件系统

    - 增强提示 

- 个性化完成任务

    - 知识库实时更新
    
    - 个人任务规划建议

### LLM 配置
在本地上手测试 Deep Agents，用本地文件读写创建 agent 并交互测试，小步验证功能，使用 longchain community 扩展自定义低成本模型 API（Moonshot、**Qwen**、Deepseek）进行调试，API_Key 配置在 .env 中，通过 python 用 load_dotenv 来导入。

#### 配置说明：.env 文件示例

在项目根目录创建 `.env` 文件，添加以下配置：

```bash
# Qwen/Tongyi（通义千问）
DASHSCOPE_API_KEY=your_dashscope_api_key_here
# 或者
QWEN_API_KEY=your_qwen_api_key_here

# Moonshot（月之暗面）
MOONSHOT_API_KEY=your_moonshot_api_key_here

# DeepSeek
DEEPSEEK_API_KEY=your_deepseek_api_key_here

# OpenAI（可选）
OPENAI_API_KEY=your_openai_api_key_here

# Anthropic Claude（可选）
ANTHROPIC_API_KEY=your_anthropic_api_key_here
```

**获取 API Key：**
- Qwen: https://dashscope.console.aliyun.com/
- Moonshot: https://platform.moonshot.cn/console/api-keys
- DeepSeek: https://platform.deepseek.com/api_keys

![image.png](attachment:image.png)

In [15]:
from langchain_community.chat_models import ChatTongyi # 使用 LangChain 社区封装的 `ChatTongyi`（Qwen）扩展模型
# from langchain_deepseek import ChatDeepSeek # `ChatDeepSeek`（Deepseek）扩展模型
# from langchain_community.chat_models.moonshot import MoonshotChat # `MoonshotChat`（Moonshot）扩展模型 （社区封装只支持对话，暂不支持兼容 deepagents ）
# 其他社区扩展模型可参考上述导入方式，社区封装缺失可参考 OpenAI 兼容
from deepagents import create_deep_agent # 使用 DeepAgents 创建智能体
from pprint import pprint # 用于美化打印输出结果

# 自动从 `.env` 加载 `DASHSCOPE_API_KEY` 或 `QWEN_API_KEY`，
import os
from dotenv import load_dotenv
load_dotenv()

# 将模型传入 `create_deep_agent`，并用 `agent.invoke` 发起一次最小请求以打印返回结果
model = ChatTongyi(model='qwen-turbo')
# model、tools、system_prompt 可根据需要调整
agent = create_deep_agent(
    model=model, 
    tools=[], 
    system_prompt='You are an assistant.')


# 最小调用示例
result = agent.invoke({"messages": [{"role": "user", "content": "你好，介绍一下自己吧。"}]})
pprint(result)

{'messages': [HumanMessage(content='你好，介绍一下自己吧。', additional_kwargs={}, response_metadata={}, id='5b2b4b5d-492c-445d-a173-c80b40555b78'),
              AIMessage(content='你好！我是你的助手，可以帮你完成各种任务。我可以使用多种工具来帮助你，比如管理待办事项、浏览文件系统、读写文件、搜索文本等。如果你有任何问题或需要帮助，请随时告诉我！', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '7da80cde-d696-4ccb-9d45-8288fac2c714', 'token_usage': {'input_tokens': 4967, 'output_tokens': 48, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 5015}}, id='lc_run--ee119398-5286-4939-b0f9-011ca75e0d34-0')]}


In [16]:
from utils import display_agent_execution, extract_final_answer # 导入展示和提取函数

display_agent_execution(result) # 展示完整执行流程
print("\n简洁答案:", extract_final_answer(result))

🤖 DeepAgents 执行流程

📝 步骤 1: 用户输入
────────────────────────────────────────────────────────────────────────────────
💬 你好，介绍一下自己吧。

🤔 步骤 2: Agent 响应
────────────────────────────────────────────────────────────────────────────────
💡 Agent 回复:
  你好！我是你的助手，可以帮你完成各种任务。我可以使用多种工具来帮助你，比如管理待办事项、浏览文件系统、读写文件、搜索文本等。如果你有任何问题或需要帮助，请随时告诉我！

✅ 执行完成


简洁答案: 你好！我是你的助手，可以帮你完成各种任务。我可以使用多种工具来帮助你，比如管理待办事项、浏览文件系统、读写文件、搜索文本等。如果你有任何问题或需要帮助，请随时告诉我！


In [17]:
answer = extract_final_answer(result)   # 只提取最终答案
print(answer)

你好！我是你的助手，可以帮你完成各种任务。我可以使用多种工具来帮助你，比如管理待办事项、浏览文件系统、读写文件、搜索文本等。如果你有任何问题或需要帮助，请随时告诉我！


 📚 DeepAgents 结果展示函数使用建议

 1. 何时使用 `display_agent_execution()`

**推荐场景:**
- 🔍 调试阶段 - 查看 Agent 的完整思考过程
- 🛠️ 工具调用 - 了解 Agent 如何使用工具
- 📊 多步任务 - 追踪复杂任务的执行流程
- 🎓 学习理解 - 学习 Agent 的决策逻辑

**示例:**
```python
result = agent.invoke({"messages": [{"role": "user", "content": "你的问题"}]})
display_agent_execution(result)  # 展示完整流程
```

2. 何时使用 `extract_final_answer()`

**推荐场景:**
- 🚀 生产环境 - 只需要最终答案
- 💬 用户界面 - 直接展示给用户
- 📝 日志记录 - 记录简洁的输出结果
- ⚡ 快速验证 - 快速检查 Agent 的回复

**示例:**
```python
result = agent.invoke({"messages": [{"role": "user", "content": "你的问题"}]})
answer = extract_final_answer(result)  # 只获取最终答案
print(answer)
```

3. 组合使用

```python
# 开发阶段:完整流程 + 最终答案
display_agent_execution(result)
print("\n简洁答案:", extract_final_answer(result))

# 生产阶段:只用最终答案
answer = extract_final_answer(result)
return answer
```

LangChain/DeepAgents兼容的消息类型

函数能够正确处理 LangChain/DeepAgents 的所有消息类型:

1. **HumanMessage** - 用户输入消息
   - 提取 `content` 属性显示用户问题
   
2. **AIMessage** - Agent 响应消息  
   - 提取 `content` 属性显示 AI 回复
   - 提取 `tool_calls` 属性显示工具调用信息
   
3. **ToolMessage** - 工具执行结果
   - 提取 `content` 属性显示工具返回结果
   - 提取 `name` 属性显示工具名称

DeepAgents 特性

✅ **内置工具**: 支持 DeepAgents 的默认工具(write_todos, write_file, read_file 等)  
✅ **自定义工具**: 支持用户自定义的工具  
✅ **工具调用链**: 能展示多步工具调用的完整流程  
✅ **消息对象**: 兼容 LangChain 的消息对象结构  
✅ **字典格式**: 同时支持字典格式的消息


### 工具、子代理自定义

In [1]:
# 加载 Obsidian 助手模块
exec(open('obsidian_assistant.py').read())

# 加载 Token 计数器
exec(open('token_counter.py').read())

# 加载展示工具
from utils import display_agent_execution, extract_final_answer

✅ Token 计数器模块加载完成


In [4]:
# 创建 Obsidian 助手
assistant = create_obsidian_assistant_v2()

# 创建 Token 计数器
token_counter = TokenCounter(model="qwen-turbo")

🔧 正在创建 Obsidian 助手 v2.0...
  ✓ 初始化模型: qwen-turbo
  ✓ 创建搜索工具（文档路径: /Users/yf/Documents/Obsidian Vault/我的知识库/Obsidian_Knowledge/obsidian-help-master）
  ✓ 配置网页搜索子代理
  ✓ 组装主代理...
✅ Obsidian 助手 v2.0 创建成功！

📋 配置信息：
  - 模型: ChatTongyi (qwen-turbo)
  - 搜索工具: search_obsidian_docs_v2（支持路径返回）
  - 子代理: web-search-agent-v2（增强触发逻辑）
  - 引用格式: Obsidian 内部链接 + 网页链接
  - 文档路径: /Users/yf/Documents/Obsidian Vault/我的知识库/Obsidian_Knowledge/obsidian-help-master



In [ ]:
# 测试问题 1
question_1 = "如何在 Obsidian 中创建双向链接？"

# 开始计时
token_counter.start_counting()

# 调用助手
result_1 = assistant.invoke({"messages": [("user", question_1)]})

# 统计 Token
record_1 = count_tokens_for_result(question_1, result_1, token_counter)
token_counter.print_current_usage(record_1)

In [ ]:
# 展示执行过程
display_agent_execution(result_1)

🤖 DeepAgents 执行流程

📝 步骤 1: 用户输入
────────────────────────────────────────────────────────────────────────────────
💬 如何在 Obsidian 中创建双向链接？

🤔 步骤 2: Agent 响应
────────────────────────────────────────────────────────────────────────────────
💡 Agent 回复:
  在 Obsidian 中创建双向链接是一种强大的功能，它允许你在一个笔记中引用另一个笔记，并在被引用的笔记中看到反向链接。以下是创建双向链接的步骤：

1. **内部链接**：使用双方括号 `[[ ]]` 包裹目标笔记的名称。例如，如果你有一个名为 "Example" 的笔记，你可以写成 `[[Example]]`。这会在当前笔记中创建一个指向 "Example" 笔记的链接（参考：[[Linking notes and files/Internal links|内部链接]]）。

2. **双向链接**：当你在某个笔记中创建一个内部链接到另一个笔记时，Obsidian 会自动在目标笔记中创建一个反向链接。这意味着在 "Example" 笔记中，你会看到一个指向原始笔记的链接（参考：[[Linking notes and files/Backlinks|反向链接]]）。

3. **使用插件**：如果你需要更高级的功能，可以使用插件如 "Backlinks" 或 "Dataview" 来增强双向链接的功能。这些插件提供了更多的自定义选项和可视化工具（参考：[[Plugins/Backlinks|Backlinks 插件]]）。

### 参考来源
⭐⭐⭐ [[Linking notes and files/Internal links|内部链接]] - 核心参考
⭐⭐ [[Linking notes and files/Backlinks|反向链接]] - 补充阅读
⭐ [[Plugins/Backlinks|Backlinks 插件]] - 补充阅读

✅ 执行完成



In [ ]:
# 只输出最终答案
print(extract_final_answer(result_1))

在 Obsidian 中创建双向链接是一种强大的功能，它允许你在一个笔记中引用另一个笔记，并在被引用的笔记中看到反向链接。以下是创建双向链接的步骤：

1. **内部链接**：使用双方括号 `[[ ]]` 包裹目标笔记的名称。例如，如果你有一个名为 "Example" 的笔记，你可以写成 `[[Example]]`。这会在当前笔记中创建一个指向 "Example" 笔记的链接（参考：[[Linking notes and files/Internal links|内部链接]]）。

2. **双向链接**：当你在某个笔记中创建一个内部链接到另一个笔记时，Obsidian 会自动在目标笔记中创建一个反向链接。这意味着在 "Example" 笔记中，你会看到一个指向原始笔记的链接（参考：[[Linking notes and files/Backlinks|反向链接]]）。

3. **使用插件**：如果你需要更高级的功能，可以使用插件如 "Backlinks" 或 "Dataview" 来增强双向链接的功能。这些插件提供了更多的自定义选项和可视化工具（参考：[[Plugins/Backlinks|Backlinks 插件]]）。

### 参考来源
⭐⭐⭐ [[Linking notes and files/Internal links|内部链接]] - 核心参考
⭐⭐ [[Linking notes and files/Backlinks|反向链接]] - 补充阅读
⭐ [[Plugins/Backlinks|Backlinks 插件]] - 补充阅读
TokenUsage(timestamp='2025-11-13 22:54:48', question='如何在 Obsidian 中创建双向链接？', prompt_tokens=21, completion_tokens=577, total_tokens=598, response_time=4.1822350025177, model='qwen-turbo', cost=0.0035039999999999997)


In [9]:
# 输出 Token 统计
print(record_1)

TokenUsage(timestamp='2025-11-13 22:54:48', question='如何在 Obsidian 中创建双向链接？', prompt_tokens=21, completion_tokens=577, total_tokens=598, response_time=4.1822350025177, model='qwen-turbo', cost=0.0035039999999999997)
